In [ ]:
# Librerías necesarias

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

from glob import glob
from tensorflow.keras.applications import InceptionV3, MobileNetV2, ResNet152, VGG16, VGG19
import sklearn.metrics as sk

from google.colab import files

# Parámetros de preprocesamiento de imágenes

image_size = (224, 224)  # Tamaño deseado de las imágenes
batch_size = 32          # Tamaño del lote para el entrenamiento

In [ ]:
!git clone https://github.com/JuliaQuero/img_dataset_parte1

Cloning into 'img_dataset_parte1'...
remote: Enumerating objects: 143690, done.
remote: Counting objects: 100% (20479/20479), done.
remote: Compressing objects: 100% (11293/11293), done.
remote: Total 143690 (delta 9383), reused 20282 (delta 9186), pack-reused 123211
Receiving objects: 100% (143690/143690), 190.15 MiB | 6.99 MiB/s, done.
Resolving deltas: 100% (10413/10413), done.
Updating files: 100% (344316/344316), done.


In [ ]:
for folder in range(1,len(glob('img_dataset_parte1/*'))+1,1):
  # Rutas de las imágenes de entrenamiento y prueba
  train_dir = 'img_dataset_parte1/V' + str(folder) + '/train'
  test_dir = 'img_dataset_parte1/V' + str(folder) + '/val'
  # Para obtener las clases de salida
  classes = glob('img_dataset_parte1/V' + str(folder) + '/train/*')
  num_classes = len(classes)

  # Generadores de datos para cargar y preprocesar las imágenes de entrenamiento y prueba
  train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)  # Normaliza los valores de píxeles entre 0 y 1
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

  test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

  # Creo txt
  out = open('V'+str(folder)+'.txt','w')

  for modelo in range(1,6,1):
    # Definimos el modelo
    base_model = InceptionV3(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
    if modelo == 2:
      base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
    elif modelo == 3:
      base_model = ResNet152(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
    elif modelo == 4:
      base_model = VGG16(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
    elif modelo == 5:
      base_model = VGG19(include_top=False, weights='imagenet', input_tensor = None, input_shape=(224, 224, 3))
    # Transfer Learning
    # Freeze convolutinal layers
    base_model.trainable=False
    x = base_model.output
    x = GlobalAveragePooling2D()(x)

    top = Dense(units=num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=top)

    # Calculo la media de 3 f1score
    f1score = 0.0

    for i in range(1,4,1):
      # Compilación y entrenamiento del modelo
      model.compile(optimizer=tf.keras.optimizers.Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
      history = model.fit(train_generator, epochs=10, validation_data=test_generator)

      # Obtenemos medidas
      Y_pred = model.predict(test_generator)
      y_pred = np.argmax(Y_pred, axis=1)
      y_true = test_generator.classes
      report = sk.classification_report(y_true, y_pred, output_dict=True)
      f1score += report['weighted avg']['f1-score']/3

    # Escribo en txt
    out.write(str(f1score)+'\n')

  # Guardo y descargo en local el txt
  out.close()
  files.download('/content/V'+str(folder)+'.txt')

Found 13114 images belonging to 5 classes.
Found 3281 images belonging to 5 classes.
87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/10
410/410 [==============================] - 59s 107ms/step - loss: 0.8573 - accuracy: 0.7169 - val_loss: 0.8440 - val_accuracy: 0.7623
Epoch 2/10
410/410 [==============================] - 39s 94ms/step - loss: 0.6705 - accuracy: 0.7892 - val_loss: 0.8198 - val_accuracy: 0.7699
Epoch 3/10
410/410 [==============================] - 39s 94ms/step - loss: 0.5987 - accuracy: 0.8190 - val_loss: 0.7934 - val_accuracy: 0.7949
Epoch 4/10
410/410 [==============================] - 41s 100ms/step - loss: 0.5557 - accuracy: 0.8375 - val_loss: 0.7581 - val_accuracy: 0.8007
Epoch 5/10
410/410 [==============================] - 38s 93ms/step - loss: 0.5239 - accuracy: 0.8438 - val_loss: 0.7469 - val_accuracy: 0.8095
Epoch 6/10
410/410 [==============================] - 38s 93ms/step - loss: 0.5015 - accuracy: 0.8487 - val_loss: 0.7503 - val_a

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


410/410 [==============================] - 149s 341ms/step - loss: 0.7299 - accuracy: 0.7574 - val_loss: 0.8745 - val_accuracy: 0.7650
Epoch 2/10
410/410 [==============================] - 137s 335ms/step - loss: 0.7189 - accuracy: 0.7632 - val_loss: 0.8831 - val_accuracy: 0.7537
Epoch 3/10
410/410 [==============================] - 137s 335ms/step - loss: 0.7094 - accuracy: 0.7656 - val_loss: 0.8796 - val_accuracy: 0.7546
Epoch 4/10
410/410 [==============================] - 137s 335ms/step - loss: 0.6994 - accuracy: 0.7695 - val_loss: 0.8785 - val_accuracy: 0.7574
Epoch 5/10
410/410 [==============================] - 138s 336ms/step - loss: 0.6914 - accuracy: 0.7712 - val_loss: 0.8685 - val_accuracy: 0.7589
Epoch 6/10
410/410 [==============================] - 137s 335ms/step - loss: 0.6832 - accuracy: 0.7730 - val_loss: 0.8778 - val_accuracy: 0.7571
Epoch 7/10
410/410 [==============================] - 152s 370ms/step - loss: 0.6755 - accuracy: 0.7770 - val_loss: 0.8467 - val_accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


410/410 [==============================] - 149s 342ms/step - loss: 0.6501 - accuracy: 0.7812 - val_loss: 0.8396 - val_accuracy: 0.7638
Epoch 2/10
410/410 [==============================] - 137s 334ms/step - loss: 0.6447 - accuracy: 0.7842 - val_loss: 0.8454 - val_accuracy: 0.7644
Epoch 3/10
410/410 [==============================] - 137s 334ms/step - loss: 0.6392 - accuracy: 0.7850 - val_loss: 0.8296 - val_accuracy: 0.7653
Epoch 4/10
410/410 [==============================] - 137s 335ms/step - loss: 0.6339 - accuracy: 0.7867 - val_loss: 0.8433 - val_accuracy: 0.7623
Epoch 5/10
410/410 [==============================] - 138s 335ms/step - loss: 0.6289 - accuracy: 0.7892 - val_loss: 0.8403 - val_accuracy: 0.7629
Epoch 6/10
410/410 [==============================] - 151s 369ms/step - loss: 0.6243 - accuracy: 0.7904 - val_loss: 0.8286 - val_accuracy: 0.7644
Epoch 7/10
410/410 [==============================] - 138s 336ms/step - loss: 0.6201 - accuracy: 0.7914 - val_loss: 0.8254 - val_accura

...

410/410 [==============================] - 88s 214ms/step - loss: 0.8219 - accuracy: 0.7340 - val_loss: 0.9393 - val_accuracy: 0.7528
Epoch 2/10
410/410 [==============================] - 83s 203ms/step - loss: 0.8163 - accuracy: 0.7351 - val_loss: 0.9331 - val_accuracy: 0.7546
Epoch 3/10
410/410 [==============================] - 83s 203ms/step - loss: 0.8107 - accuracy: 0.7359 - val_loss: 0.9244 - val_accuracy: 0.7507
Epoch 4/10
410/410 [==============================] - 83s 204ms/step - loss: 0.8052 - accuracy: 0.7369 - val_loss: 0.9210 - val_accuracy: 0.7513
Epoch 5/10
410/410 [==============================] - 88s 215ms/step - loss: 0.8004 - accuracy: 0.7379 - val_loss: 0.9184 - val_accuracy: 0.7553
Epoch 6/10
410/410 [==============================] - 84s 204ms/step - loss: 0.7952 - accuracy: 0.7387 - val_loss: 0.9170 - val_accuracy: 0.7580
Epoch 7/10
410/410 [==============================] - 84s 205ms/step - loss: 0.7905 - accuracy: 0.7394 - val_loss: 0.9152 - val_accuracy: 0.7

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Found 13114 images belonging to 5 classes.
Found 3281 images belonging to 5 classes.
Epoch 1/10
410/410 [==============================] - 45s 99ms/step - loss: 0.8941 - accuracy: 0.7071 - val_loss: 0.8664 - val_accuracy: 0.7531
Epoch 2/10
410/410 [==============================] - 39s 96ms/step - loss: 0.7138 - accuracy: 0.7593 - val_loss: 0.8209 - val_accuracy: 0.7681
Epoch 3/10
410/410 [==============================] - 39s 95ms/step - loss: 0.6373 - accuracy: 0.7927 - val_loss: 0.7943 - val_accuracy: 0.7824
Epoch 4/10
410/410 [==============================] - 39s 95ms/step - loss: 0.5900 - accuracy: 0.8145 - val_loss: 0.7659 - val_accuracy: 0.7934
Epoch 5/10
410/410 [==============================] - 41s 101ms/step - loss: 0.5577 - accuracy: 0.8246 - val_loss: 0.7696 - val_accuracy: 0.7918
Epoch 6/10
410/410 [==============================] - 39s 95ms/step - loss: 0.5311 - accuracy: 0.8329 - val_loss: 0.7613 - val_accuracy: 0.7906
Epoch 7/10
410/410 [==============================